# VacationPy
----

#### Note
* Keep an eye on API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor usage and billing.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from dotenv import load_dotenv

# API key setup
load_dotenv()

google_api_key = os.getenv("google_api_key")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/weather_data.csv")
del weather_df['Unnamed: 0'] # remove previous index column
weather_df

,Name,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,victoria,22.2855,114.1577,82.36,74.0,73.0,17.22,HK,1.630055e+09
1,lebu,-37.6167,-73.6500,46.56,87.0,100.0,10.29,CL,1.630055e+09
2,chapais,49.7834,-74.8492,51.58,83.0,100.0,8.28,CA,1.630055e+09
3,hobart,-42.8794,147.3294,48.06,82.0,75.0,5.75,AU,1.630055e+09
4,albany,42.6001,-73.9662,72.81,92.0,18.0,3.67,US,1.630055e+09
...,...,...,...,...,...,...,...,...,...
565,uray,-24.7000,44.7500,86.68,21.0,0.0,10.87,MG,1.630055e+09
566,punta de bombon,29.5667,-104.4167,83.41,44.0,75.0,9.22,MX,1.630055e+09
567,mabaruma,-9.4431,147.1797,78.04,88.0,56.0,11.83,PG,1.630055e+09
568,macau,6.6592,7.7596,78.30,85.0,100.0,1.30,NG,1.630055e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
locations = weather_df[['Latitude','Longitude']]
humidity = weather_df['Humidity']

fig = gmaps.figure(zoom_level=1.6, center=[0, 0])
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = weather_df[(weather_df['Max_Temp']<80) &
                      (weather_df['Max_Temp']>70) &
                      (weather_df['Wind_Speed']<10) &
                      (weather_df['Cloudiness']==0)]
hotel_df['Hotel_Name'] = ""
hotel_df = hotel_df.reset_index(drop=True)
hotel_df = hotel_df.rename(columns={"Name":"City"})
hotel_df['City'] = hotel_df['City'].str.title()
hotel_df

C:\Users\srobi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
0,Los Llanos De Aridane,-23.7000,133.8833,78.35,11.0,0.0,0.00,AU,1.630055e+09,
1,Havre-Saint-Pierre,39.1207,27.1805,79.86,21.0,0.0,8.88,TR,1.630055e+09,
2,Ponta Do Sol,42.7762,-9.0603,72.10,73.0,0.0,4.61,ES,1.630055e+09,
3,Valparaiso,-20.7333,139.5000,74.97,10.0,0.0,5.75,AU,1.630055e+09,
4,Sungaipenuh,4.9333,-52.3333,75.24,92.0,0.0,3.44,GF,1.630055e+09,
5,Margate,44.8324,-0.5141,70.92,68.0,0.0,6.91,FR,1.630055e+09,
6,Turan,39.1502,-123.2078,73.04,67.0,0.0,3.29,US,1.630055e+09,


In [11]:
# find the first hotel result for city within 5,000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get latitude and longitude from dataframe
    latitude = row['Latitude']
    longitude = row['Longitude']
    city_name = row['City']

    params = {
        "location": f"{latitude}, {longitude}",
        "radius": "5000",
        "type": "hotel",
        "key": google_api_key,
    }

    # assemble url and make API request
    print(f"Retrieving results for index {index}: {city_name.title()}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"For city {city_name.title()} we found hotel {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel_Name'] = results[0]['name']
         
    except (KeyError, IndexError):
        print(f"Missing for city {city_name} result... skipping.")
        
    print("-"*64)

Retrieving results for index 0: Los Llanos De Aridane
For city Los Llanos De Aridane we found hotel Alice Springs.
----------------------------------------------------------------
Retrieving results for index 1: Havre-Saint-Pierre
For city Havre-Saint-Pierre we found hotel Bergama.
----------------------------------------------------------------
Retrieving results for index 2: Ponta Do Sol
For city Ponta Do Sol we found hotel Muros.
----------------------------------------------------------------
Retrieving results for index 3: Valparaiso
For city Valparaiso we found hotel Mount Isa.
----------------------------------------------------------------
Retrieving results for index 4: Sungaipenuh
For city Sungaipenuh we found hotel Cayenne.
----------------------------------------------------------------
Retrieving results for index 5: Margate
For city Margate we found hotel Bordeaux.
----------------------------------------------------------------
Retrieving results for index 6: Turan
For c

In [14]:
hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
0,Los Llanos De Aridane,-23.7000,133.8833,78.35,11.0,0.0,0.00,AU,1.630055e+09,Alice Springs
1,Havre-Saint-Pierre,39.1207,27.1805,79.86,21.0,0.0,8.88,TR,1.630055e+09,Bergama
2,Ponta Do Sol,42.7762,-9.0603,72.10,73.0,0.0,4.61,ES,1.630055e+09,Muros
3,Valparaiso,-20.7333,139.5000,74.97,10.0,0.0,5.75,AU,1.630055e+09,Mount Isa
4,Sungaipenuh,4.9333,-52.3333,75.24,92.0,0.0,3.44,GF,1.630055e+09,Cayenne
5,Margate,44.8324,-0.5141,70.92,68.0,0.0,6.91,FR,1.630055e+09,Bordeaux
6,Turan,39.1502,-123.2078,73.04,67.0,0.0,3.29,US,1.630055e+09,Ukiah


In [15]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt style="color: black; font-weight: 800; font-size: 15px">Name</dt>
<dd>{Hotel_Name}</dd>
<dt style="color: black; font-weight: 800; font-size: 15px">City</dt>
<dd>{City}</dd>
<dt style="color: black; font-weight: 800; font-size: 15px">Country</dt>
<dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [16]:
locations2

,Latitude,Longitude
0,-23.7000,133.8833
1,39.1207,27.1805
2,42.7762,-9.0603
3,-20.7333,139.5000
4,4.9333,-52.3333
5,44.8324,-0.5141
6,39.1502,-123.2078


In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations2,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))